# Recipe Pipeline
Configure your paths & regex flags below, then run all cells ▶️

In [ ]:
# @title ⚙️ Pipeline Parameters
repo_url = "https://github.com/Thireus/GGUF-Tool-Suite.git"         #@param {type:"string"}
model_name = "recipe_examples/DeepSeek-R1-0528"                     #@param {type:"string"}
model_link = "https://huggingface.co/deepseek-ai/DeepSeek-R1-0528"  #@param {type:"string"}
csv_file = "ppl_results.csv"                                        #@param {type:"string"}

# regex lists (as Python lists of strings)
gpu_tensors = [r".*",]                                                                                                                                                           #@param {type:"raw"}
cpu_tensors = [r"blk\.([3-9]|[1-5][0-9]|60)\.ffn_down_exps\.weight", r"blk\.([3-9]|[1-5][0-9]|60)\.ffn_up_exps\.weight", r"blk\.([3-9]|[1-5][0-9]|60)\.ffn_gate_exps\.weight",]  #@param {type:"raw"}

# quant types
cpu_quants = ["iq4_ks", "iq3_k", "iq2_k", "iq1_m_r4"]   #@param {type:"raw"}
gpu_quants = ["q8_0", "iq5_k_r4", "iq6_k"]              #@param {type:"raw"}

# sizes & tuning
cpu_tensors_max_size = 230      #@param {type:"integer"}
gpu_tensors_max_size = "95%"    #@param {type:"string"}
tolerance = 0.01                #@param {type:"number"}
exponential_factor = 8          #@param {type:"integer"}

# assignment override
gpu_assign_qtype = "iq4_xs"                                                 #@param {type:"string"}
gpu_assign_tensors = [r"blk\.([0-9]|[1-5][0-9]|60)\.attn_k_b\.weight=q8_0",] #@param {type:"raw"}


In [ ]:
# Clone and cd into the tool suite
!git clone {repo_url} && cd GGUF-Tool-Suite

In [ ]:
# Build up all the CLI flags from our params
import shlex, subprocess

def join_list(flag, items):
    return " ".join(f"--{flag} {shlex.quote(it)}" for it in items)

base_cmd = [
    "python", "quant_assign.py", csv_file,
]
flags = []
flags += [f"--cpu-tensors-max-size {cpu_tensors_max_size}"]
flags += [f"--gpu-tensors-max-size {gpu_tensors_max_size}"]
flags += [f"--tolerance {tolerance}"]
flags += [f"--exponential-factor {exponential_factor}"]

flags += join_list("gpu-tensors", gpu_tensors).split()
flags += join_list("cpu-tensors", cpu_tensors).split()
flags += join_list("cpu-quants", cpu_quants).split()
flags += join_list("gpu-quants", gpu_quants).split()
flags += [f"--gpu-assign-qtype {gpu_assign_qtype}"]
flags += join_list("gpu-assign-tensors", gpu_assign_tensors).split()

# run quant_assign.py and pipe into merger script
cmd = base_cmd + flags

# 1. Run quant_assign.py and capture its stdout
result = subprocess.run(cmd, capture_output=True, text=True)
if result.returncode != 0:
    print("quant_assign.py failed:", result.stderr)
    raise SystemExit(1)

# 2. Feed that output into quants_regex_merger.sh
merge_cmd = [
    "bash", "./quants_regex_merger.sh",
    "--model-name", model_name,
    "--add-ppl", "0",
    "--model-link", model_link
]
merge = subprocess.run(merge_cmd, input=result.stdout, text=True)
if merge.returncode != 0:
    print("quants_regex_merger.sh failed")
    raise SystemExit(1)